In [1]:
#preamble
import pandas as pd
import numpy as np
import matplotlib as mt
import glob as glob

#downloading files
filenames = glob.glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k]*160c.dta')
list_of_all_dfs  = [pd.read_stata(f) for f in filenames]

#pulling out variables of interest 
prev = [list_of_all_dfs[i].filter(regex='jbhrcpr$') for i in range(11)]
sex = [list_of_all_dfs[i].filter(regex='hgsex$') for i in range(11)]
job = [list_of_all_dfs[i].filter(regex='esbrd$') for i in range(11)]
hrs_act = [list_of_all_dfs[i].filter(regex='jbhruc$') for i in range(11)]
hrs_id = [list_of_all_dfs[i].filter(regex='jbtprhr$') for i in range(11)]

# making into dataframes 
prev = [pd.DataFrame(prev[i]) for i in range(11)]
sex = [pd.DataFrame(sex[i]) for i in range(11)]
job = [pd.DataFrame(job[i]) for i in range(11)]
hrs_act = [pd.DataFrame(hrs_act[i]) for i in range(11)]
hrs_id = [pd.DataFrame(hrs_id[i]) for i in range(11)]

In [2]:
# merging into one dataframe for simplicity 
result1 = [pd.merge(sex[i], job[i], left_index=True, right_index=True) for i in range(11)]
result1 = [pd.merge(result1[i], prev[i], left_index=True, right_index=True) for i in range(11)]
result1 = [pd.merge(result1[i], hrs_id[i], left_index=True, right_index=True) for i in range(11)]
result1 = [pd.merge(result1[i], hrs_act[i], left_index=True, right_index=True) for i in range(11)]

In [3]:
# narrowing results 
# this is now all employed respondants that want more/less hours 
pt = [result1[i][result1[i].iloc[:, 1] == '[1] Employed'] for i in range(11)]
pt = [pt[i][(pt[i].iloc[:, 2] == '[3] More hours') | (pt[i].iloc[:, 2] == '[1] Fewer hours')] for i in range(11)]
# narrowing to just men 
men = [pt[i][pt[i].iloc[:, 0] == '[2] Female'] for i in range(11)]

In [9]:
#pulling out columns of interest 
des1 = [men_few[i].filter(regex='jbtprhr$') for i in range(11)]
act1 = [men_few[i].filter(regex='jbhruc$') for i in range(11)]

merged = [pd.merge(des1[i], act1[i], left_index=True, right_index=True) for i in range(11)]
merged1 = [pd.DataFrame(merged[i]) for i in range(11)]


# pulling out important variables, making numeric
des11 = [merged1[i].iloc[:,0] for i in range(11)]
act11 = [merged1[i].iloc[:,1] for i in range(11)]
numb_des = [pd.to_numeric(des11[i], errors = 'coerce') for i in range(11)]
numb_act = [pd.to_numeric(act11[i], errors = 'coerce') for i in range(11)]

#merging numeric columns, then dropping na responses 
check = [pd.merge(numb_des[i], numb_act[i], left_index=True, right_index=True) for i in range(11)]
check_dropped = [check[i].dropna() for i in range(11)]
few_n = [len(check_dropped[i]) for i in range(11)]

#splitting back into seperate columns
new_des = [check_dropped[i].iloc[:,0] for i in range(11)]
new_act = [check_dropped[i].iloc[:,1] for i in range(11)]

#getting the mean of desired hours each wave 
des_mean = [new_des[i].mean() for i in range(11)]
act_mean = [new_act[i].mean() for i in range(11)]
des_total = [len(numb_des[i]) for i in range(11)]


#creating difference column
diff_few = [des_mean[i] - act_mean[i] for i in range(11)]
diff_few1 = pd.DataFrame([diff_few[0], diff_few[1], diff_few[2], diff_few[3], diff_few[4], diff_few[5], diff_few[6], diff_few[7], diff_few[8], diff_few[9], diff_few[10] ])
diff_few1 = round(diff_few1, 1)


dip = pd.DataFrame(diff_few1.mean())
dip= round(dip, 1)

#inputting total at end of column 
diff_few_final = pd.concat([diff_few1, dip], axis=0)

#creating totals column
few_totals = pd.DataFrame([few_n[0], few_n[1], few_n[2], few_n[3], few_n[4], few_n[5], few_n[6], few_n[7], few_n[8], few_n[9], few_n[10]])
few_totals11 = pd.DataFrame(few_totals.sum())
few_totals_final = pd.concat([few_totals, few_totals11], axis=0)
few_table = pd.merge(few_totals_final, diff_few_final, left_index=True, right_index=True)

#joining together
few_table = pd.concat([few_totals_final, diff_few_final], axis=1)


In [10]:
# looking at those who prefer to work more 
men_mo = [men[i][men[i].iloc[:, 2] == '[3] More hours'] for i in range(11)]

# putting into columns of interest 
des_mo = [men_mo[i].filter(regex='jbtprhr$') for i in range(11)]
act_mo = [men_mo[i].filter(regex='jbhruc$') for i in range(11)]

#merging into one frame, turning into a dataframe
mo_merged = [pd.merge(des_mo[i], act_mo[i], left_index=True, right_index=True) for i in range(11)]
mo_merged1 = [pd.DataFrame(mo_merged[i]) for i in range(11)]

#pulling out important variables, making numeric

numb_des = [mo_merged1[i].iloc[:,0] for i in range(11)]
numb_act = [mo_merged1[i].iloc[:,1] for i in range(11)]
numb_des = [pd.to_numeric(numb_des[i], errors = 'coerce') for i in range(11)]
numb_act = [pd.to_numeric(numb_act[i], errors = 'coerce') for i in range(11)]

#merging numeric columns, then dropping na responses 
checkmo = [pd.merge(numb_des[i], numb_act[i], left_index=True, right_index=True) for i in range(11)]
checkmo_dropped = [checkmo[i].dropna() for i in range(11)]
mo_n = [len(checkmo_dropped[i]) for i in range(11)]

#splitting back into seperate columns
new_desmo = [checkmo_dropped[i].iloc[:,0] for i in range(11)]
new_actmo = [checkmo_dropped[i].iloc[:,1] for i in range(11)]



#getting the mean of desired and actual hours each wave 
des_meanmo = [new_desmo[i].mean() for i in range(11)]
act_meanmo = [new_actmo[i].mean() for i in range(11)]

#creating difference column
diff_mo = [des_meanmo[i] - act_meanmo[i] for i in range(11)]
diff_mo1 = pd.DataFrame([diff_mo[0], diff_mo[1], diff_mo[2], diff_mo[3], diff_mo[4], diff_mo[5], diff_mo[6], diff_mo[7], diff_mo[8], diff_mo[9], diff_mo[10] ])
diff_mo1 = round(diff_mo1, 1)

dipmo = pd.DataFrame(diff_mo1.mean())
dipmo= round(dipmo, 1)


#inputting total at end of column 
diff_mo_final = pd.concat([diff_mo1, dipmo], axis=0)


#now to do the same for the number of workers to get totals column

mo_totals = pd.DataFrame([mo_n[0], mo_n[1], mo_n[2], mo_n[3], mo_n[4], mo_n[5], mo_n[6], mo_n[7], mo_n[8], mo_n[9], mo_n[10]])
mo_sum = pd.DataFrame(mo_totals.sum())
mo_totals_final = pd.concat([mo_totals, mo_sum], axis=0)
mo_table = pd.concat([mo_totals_final, diff_mo_final], axis=1)

In [11]:
# Creating a table for the average of any mismatch 

#pulling out variables of interest 
des_all = [men[i].filter(regex='jbtprhr$') for i in range(11)]
act_all = [men[i].filter(regex='jbhruc$') for i in range(11)]

#merging into one 
all_merged = [pd.merge(des_all[i], act_all[i], left_index=True, right_index=True) for i in range(11)]

#dropping non-responses 
drop_all = [all_merged[i].dropna() for i in range(11)]
des_ = [drop_all[i].iloc[:,0] for i in range(11)]
des_all1 = [pd.to_numeric(des_[i], errors = 'coerce') for i in range(11)]

# getting the mean 
#des_allmean = [des_all1[i].mean() for i in range(11)]
des_alltot = [len(des_all1[i]) for i in range(11)]

act_ = [drop_all[i].iloc[:,1] for i in range(11)]

# turning into numbers, getting mean and total 
act_all1 = [pd.to_numeric(act_[i], errors = 'coerce') for i in range(11)]
act_allmean = [act_all1[i].mean() for i in range(11)]
#act_alltot = [len(act_all1[i]) for i in range(11)]

#merging the two numerical columns, then dropping na values for both columns
test = [pd.merge(des_all1[i], act_all1[i], left_index=True, right_index=True) for i in range(11)]
test1 = [test[i].dropna() for i in range(11)]
act_alltot = [len(test1[i]) for i in range(11)]
#should now be left with the correct amount of responses
# will split into seperate columns again

des_all = [test1[i].iloc[:,0] for i in range(11)]
act_all = [test1[i].iloc[:,1] for i in range(11)]

# getting means 
act_allmean = [act_all[i].mean() for i in range(11)]
des_allmean = [des_all[i].mean() for i in range(11)]


#getting difference, creating column
diff_all = [des_allmean[i] - act_allmean[i] for i in range(11)]
diff_all1=  pd.DataFrame([diff_all[0], diff_all[1], diff_all[2], diff_all[3], diff_all[4], diff_all[5], diff_all[6], diff_all[7], diff_all[8], diff_all[9], diff_all[10]])
diff_all1= round(diff_all1, 1)

#getting mean of difference, popping in on the bottom of column
diff_mean = pd.DataFrame(diff_all1.mean())
diff_mean = round(diff_mean, 1)
diff_all_final = pd.concat([diff_all1, diff_mean])

#making a column for the totals 
all_totals = pd.DataFrame([act_alltot[0], act_alltot[1], act_alltot[2], act_alltot[3], act_alltot[4], act_alltot[5], act_alltot[6], act_alltot[7], act_alltot[8], act_alltot[9], act_alltot[10]])
all_sum = all_totals.sum()
all_final = pd.concat([all_totals, all_sum], axis = 0)


#creating the final table 
final_table = pd.concat([all_final, diff_all_final], axis=1)

In [12]:
# joining my three mini-tables into one table, labelling index 
table5 = pd.concat([few_table, mo_table, final_table], axis=1)
table5.index= [ '2001 (1)', '2002 (2)', '2003 (3)', '2004 (4)', '2005 (5)', '2006 (6)', '2007 (7)', '2008 (8)', '2009 (9)', '2010 (10)', '2011 (11)', 'All']

In [13]:
table5.columns = ['Number of workers', 'Average hours', 'Number of workers', 'Average hours', 'Number of workers', 'Average hours']

In [14]:
table5

,Number of workers,Average hours,Number of workers,Average hours,Number of workers,Average hours
2001 (1),1059,-13.3,682,11.7,1741,-3.6
2002 (2),926,-13.2,664,11.8,1590,-2.7
2003 (3),941,-13.4,648,11.5,1589,-3.2
2004 (4),948,-13.0,597,11.4,1545,-3.6
2005 (5),1033,-13.2,667,11.0,1700,-3.7
2006 (6),1044,-12.9,611,10.6,1655,-4.2
2007 (7),1075,-13.3,619,10.6,1694,-4.6
2008 (8),1013,-13.0,585,11.1,1598,-4.2
2009 (9),971,-13.3,636,10.7,1607,-3.8
2010 (10),1016,-13.2,690,10.8,1706,-3.5
